### イメージを読み込む

In [ ]:
import numpy  as np
import os
import glob 
import re

import matplotlib.pyplot as plt

from PIL import Image

%matplotlib inline

In [ ]:
#sprint18_image

path = './sprint18_image/training/' 
flist = os.listdir(path)

In [ ]:
!ls -a .//sprint18_image/training/

In [ ]:
flist

In [ ]:
# 不可視ファイルの.DS_Storeファイルを除いて読み込む

'''
余談

.DS_Storeファイルとは？ 開けるの？

https://miloserdov.org/?p=3867

'''

flist_ignore = [name for name in os.listdir(path) if not name.startswith('.')]
flist_ignore

In [ ]:
img_list = glob.glob(path + '/*' + ".jpg")
img_list

### イメージのロード、配列化、リサイズ、データセット作成

In [ ]:
# np.resizeはだめ、ぜったい

dog_img_array = np.empty((0,224,224,3))
cat_img_array = np.empty((0,224,224,3))

for img in img_list:
    
    # ファイル名に'dog'が含まれるイメージ
    if re.search('dog', img):
        
        dog_img_ = Image.open(img)
        
        # サイズを揃える
        dog_img_ = dog_img_.resize((224, 224))
        
        # PIL.Image.Imageからnumpy配列へ
        dog_img = np.array(dog_img_)
        
        # 正規化
        dog_img = dog_img / 255.
        
        # axisの追加
        dog_img = dog_img.reshape((1,224,224,3))
        
        dog_img_array = np.concatenate([dog_img_array, dog_img], axis = 0)
        
        dog_img_.close()
    
    # ファイル名に'cat'が含まれるイメージ
    if re.search('cat', img):
        
        cat_img_ = Image.open(img)
        
        cat_img_ = cat_img_.resize((224, 224))
        
        cat_img = np.array(cat_img_)
        
        cat_img = cat_img / 255.
        
        cat_img = cat_img.reshape((1,224,224,3))
        
        cat_img_array = np.concatenate([cat_img_array, cat_img], axis = 0)
        
        cat_img_.close()

In [ ]:
print('dog_image:{}  cat_image:{}'.format(dog_img_array.shape, cat_img_array.shape))

### イメージの出力

In [ ]:
# 配列のまま出力

print('データ型:', cat_img_array[3].dtype)

cat_img_array[3]

In [ ]:
# 配列を画像として出力

#plt.rcParams['figure.figsize'] = (5.0, 5.0)

#plt.imshow(cat_img_array[1])

#plt.show()

In [ ]:
#'''

#画像の中心を切り出し

#https://note.nkmk.me/python-pillow-image-crop-trimming/

#'''


#def crop_center(pil_img, crop_width, crop_height):
    
#   img_width, img_height = pil_img.size
    
#    return pil_img.crop(((img_width - crop_width) // 2,
#                         (img_height - crop_height) // 2,
#                         (img_width + crop_width) // 2,
#                         (img_height + crop_height) // 2))


In [ ]:
#img = Image.open(img_list[1])

#img_new = crop_center(img, 224, 224)

#print(type(img_new))

#plt.imshow(img_new)

#plt.show()

#img.close()

## 【問題1】自作データセットでの分類の学習

In [ ]:
import numpy as np
#import cv2

In [ ]:
#plt.imshow(cat_img_array[2])

In [ ]:
# 結合してX_trainに

X_train = np.concatenate((cat_img_array, dog_img_array), axis=0)
X_train.shape

In [ ]:
# ラベル作成 0=cat, 1=dog

y_train = np.array((0,0,0,0,0,1,1,1,1,1))
#y_train = y_train.reshape(-1, 1)
y_train.shape

In [ ]:
# One-hot
#from sklearn.preprocessing import OneHotEncoder

#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
#y_train = enc.fit_transform(y_train)
#y_train

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
import tensorflow as tf

In [ ]:
conv_base = tf.keras.applications.vgg16.VGG16(
                                                                        weights='imagenet', 
                                                                        include_top=False, 
                                                                        input_shape=(224, 224, 3)
                                                                        )

In [ ]:
#from tf.keras import models

model = tf.keras.models.Sequential()

model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                             batch_size=1,
                             epochs=10,
                             verbose=1,  # ０=非表示、2=エポックごとの表示
                             validation_data=(X_val, y_val)
                             )

## 【問題2】分類データセットに対するデータ拡張

In [ ]:
pip list

In [ ]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

In [ ]:
def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [ ]:
 transform = A.Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.5),
        OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.5),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.5),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),
        ], p=0.),
        HueSaturationValue(p=0.3),
    ])


In [ ]:
# データ拡張

image = cv2.imread('./sprint18_image/training/cat.01.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#visualize(image)

image.shape
augmented_image = transform(image=image)['image']
#visualize(augmented_image)

In [ ]:
# 猫画像を水増しする

#aug_num = 10

#for j in range(5):
    
#    image = cv2.imread('./sprint18_image/training/cat.0{}.jpg'.format(j+1))

#    for i in range(aug_num):
        
#        augmented_image = transform(image=image)['image']
        # 可視化する場合
        #show_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
        #visualize(show_image)

#        cv2.imwrite("./sprint18_image/aug_train/cat.{}{}.jpg".format(j, i), augmented_image)

In [ ]:
# 犬画像を水増しする

#aug_num = 10

#for j in range(5):
    
#    image = cv2.imread('./sprint18_image/training/dog.0{}.jpg'.format(j+1))

#    for i in range(aug_num):
        
#        augmented_image = transform(image=image)['image']
        #show_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
        #visualize(show_image)

#        cv2.imwrite("./sprint18_image/aug_train/dog.{}{}.jpg".format(j, i), augmented_image)

## 【問題3】物体検出データセットの用意

### Define functions to visualize bounding boxes and class labels on an image

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    
    """Visualizes a single bounding box on the image"""
    #x_min, y_min, w, h = bbox
    x_min, y_min, x_max, y_max = bbox
    x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
    #x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    
    cv2.putText(
                            img,
                            text=class_name,
                            org=(x_min, y_min - int(0.3 * text_height)),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=0.35, 
                            color=TEXT_COLOR, 
                            lineType=cv2.LINE_AA,
                            )
    
    return img



def visualize(image, bboxes, category_ids, category_id_to_name):
    
    img = image.copy()
    
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
        
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)
    
    return img

### Get an image and annotations for it

In [ ]:
# Load the image from the disk

image = cv2.imread('./sprint18_image/training/cat.01.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#plt.imshow(image)

### xmlデータ構造をパース（分析）して座標情報を獲得する

In [ ]:
import os
import xml.etree.ElementTree as ET


# #読み込み例
# tree = ET.parse(path + 'cat.41box.xml')
# #一番上の階層
# root = tree.getroot()

COORDINATE = 4

path = './sprint18_image/training_bbox_voc/'
boxbox = []
point = COORDINATE

for filename in os.listdir(path):
    
    boundbox = []    
    if not filename.endswith('.xml'): 
        continue
        
    fullname = os.path.join(path, filename)
    tree = ET.parse(fullname)
    
    bndbox = tree.findall('object/bndbox')
    
    for i in range(point):
        # 座標取得
        boundbox.append(bndbox[0][i].text)
    boxbox.append(boundbox)
    
boxbox

In [ ]:
# Define two bounding boxes with coordinates and class labels

bboxes = [[160, 13, 360, 360]]
category_ids = [0]

# We will use the mapping from category_id to the class name
# to visualize the class label for the bounding box on the image
category_id_to_name = {0: 'cat', 1: 'dog'}

In [ ]:
# Visuaize the original image with bounding boxes

#visualize(image, bboxes, category_ids, category_id_to_name)

## Sprint18 [問題4] 物体検出用データ拡張

In [ ]:
transform = A.Compose(
    [A.HorizontalFlip(p=0.5)],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
)

In [ ]:
#random.seed(7)
#transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#img = visualize(
#    transformed['image'],
#    transformed['bboxes'],
#    transformed['category_ids'],
#    category_id_to_name,
#)

In [ ]:
transform = A.Compose(
                                     [A.HorizontalFlip(p=0.5),
                                     RandomRotate90(),
                                     Transpose(),
                                      
                                     OneOf([
                                                 IAAAdditiveGaussianNoise(),
                                                 GaussNoise(),
                                                 ], p=0.5),
                                      
                                    OneOf([
                                                MotionBlur(p=0.2),
                                                MedianBlur(blur_limit=3, p=0.1),
                                                Blur(blur_limit=3, p=0.1),
                                                ], p=0.5),
                                      
                                    OneOf([
                                                #OpticalDistortion(p=0.3),
                                                #GridDistortion(p=0.1),
                                                IAAPiecewiseAffine(p=0.3),
                                                ], p=0.5),
                                    
                                    OneOf([
                                                CLAHE(clip_limit=2),
                                                IAASharpen(),
                                                IAAEmboss(),
                                                RandomBrightnessContrast(),
                                                ], p=0.),
                                    
                                    HueSaturationValue(p=0.3), 
                                    ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
                                    ],
                                    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
                                    )

In [ ]:
boxbox

In [ ]:
# データサイズが巨大になるため、以下の処理はコメントアウト

# bboxつき猫画像を水増しする
# cat01

#random.seed()
#aug_num = 10
#category_id_to_name = {0: 'cat', 1: 'dog'}

#bboxes = [[160, 13, 360, 360]]
#category_ids = [0] #cat

#image = cv2.imread('./sprint18_image/training/cat.01.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/cat.{0:02d}.jpg".format(i), img)

In [ ]:
# データサイズが巨大になるため、以下の処理はコメントアウト

# cat02

#bboxes = [[177, 192, 815, 971]]
#category_ids = [0] #cat
#
#image = cv2.imread('./sprint18_image/training/cat.02.jpg')
#
#for i in range(aug_num):
#        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )
#    
#    cv2.imwrite("./sprint18_image/aug_train_bbox/cat.1{}.jpg".format(i), img)

In [ ]:
# cat03

#bboxes = [[322, 7, 693, 490]]
#category_ids = [0] #cat

#image = cv2.imread('./sprint18_image/training/cat.03.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#   img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/cat.2{}.jpg".format(i), img)

In [ ]:
# cat04

#bboxes =  [[380, 237, 2396, 1702]]
#category_ids = [0] #cat

#image = cv2.imread('./sprint18_image/training/cat.04.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/cat.3{}.jpg".format(i), img)

In [ ]:
# cat05

#bboxes =  [[227, 64, 690, 533]]
#category_ids = [0] #cat

#image = cv2.imread('./sprint18_image/training/cat.05.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/cat.4{}.jpg".format(i), img)

In [ ]:
# dog01

#bboxes = [[128, 66, 427, 312]]
#category_ids = [1] #dog

#image = cv2.imread('./sprint18_image/training/dog.01.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/dog.{0:02d}.jpg".format(i), img)

In [ ]:
# dog02

#bboxes = [[115, 24, 1202, 974]]
#category_ids = [1] #dog

#image = cv2.imread('./sprint18_image/training/dog.02.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/dog.1{}.jpg".format(i), img)

In [ ]:
# dog03

#bboxes = [[182, 72, 715, 509]]
#category_ids = [1] #dog

#image = cv2.imread('./sprint18_image/training/dog.03.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/dog.2{}.jpg".format(i), img)

In [ ]:
# dog04

#bboxes = [[458, 61, 862, 759]]
#category_ids = [1] #dog

#image = cv2.imread('./sprint18_image/training/dog.04.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/dog.3{}.jpg".format(i), img)

In [ ]:
# dog05

#bboxes = [[182, 20, 917, 576]]
#category_ids = [1] #dog

#image = cv2.imread('./sprint18_image/training/dog.05.jpg')

#for i in range(aug_num):
        
#    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
#    img = visualize(
#                             transformed['image'],
#                             transformed['bboxes'],
#                             transformed['category_ids'],
#                             category_id_to_name,
#                             )

#    cv2.imwrite("./sprint18_image/aug_train_bbox/dog.4{}.jpg".format(i), img)

### 学習

In [ ]:
path = './sprint18_image/aug_train_bbox/'
flist = os.listdir(path)
flist

In [ ]:
img_list = glob.glob(path + '/*' + ".jpg")
img_list

In [ ]:
dog_img_array = np.empty((0,224,224,3))
cat_img_array = np.empty((0,224,224,3))

for img in img_list:
    
    if re.search('dog', img):
        
        dog_img_ = Image.open(img)
        dog_img_ = dog_img_.resize((224, 224))
        dog_img = np.array(dog_img_)
        dog_img = dog_img / 255.
        dog_img = dog_img.reshape((1,224,224,3))
        dog_img_array = np.concatenate([dog_img_array, dog_img], axis = 0)
        dog_img_.close()
    
    if re.search('cat', img):
        
        cat_img_ = Image.open(img)
        cat_img_ = cat_img_.resize((224, 224))
        cat_img = np.array(cat_img_)
        cat_img = cat_img / 255.
        cat_img = cat_img.reshape((1,224,224,3))
        cat_img_array = np.concatenate([cat_img_array, cat_img], axis = 0)
        cat_img_.close()

In [ ]:
print('dog_image:{}  cat_image:{}'.format(dog_img_array.shape, cat_img_array.shape))

In [ ]:
# 結合してX_trainに

X_train = np.concatenate((cat_img_array, dog_img_array), axis=0)
X_train.shape

In [ ]:
# ラベル作成 0=cat, 1=dog

label_cat = np.zeros((len(cat_img_array)))
label_cat

In [ ]:
label_dog = np.ones((len(dog_img_array)))
label_dog

In [ ]:
y_train = np.concatenate((label_cat, label_dog))
#y_train = y_train.reshape(-1, 1)
y_train.shape

In [ ]:
# One-hot
#from sklearn.preprocessing import OneHotEncoder

#enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
#y_train = enc.fit_transform(y_train)
#y_train

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
conv_base = tf.keras.applications.vgg16.VGG16(
                                                                        weights='imagenet', 
                                                                        include_top=False, 
                                                                        input_shape=(224, 224, 3)
                                                                        )

In [ ]:
#from tf.keras import models

model = tf.keras.models.Sequential()

model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                             batch_size=1,
                             epochs=10,
                             verbose=1,  # ０=非表示、2=エポックごとの表示
                             validation_data=(X_val, y_val)
                             )

### test

In [ ]:
path = './sprint18_image/testdataset/'
flist = os.listdir(path)
flist

# チャネル数４のデータは排除

In [ ]:
img_list = glob.glob(path + '/*' + ".jpg")
img_list

In [ ]:
dog_img_array = np.empty((0,224,224,3))
cat_img_array = np.empty((0,224,224,3))

for img in img_list:
    
    if re.search('dog', img):
        
        dog_img_ = Image.open(img)
        dog_img_ = dog_img_.resize((224, 224))
        dog_img = np.array(dog_img_)
        #print(dog_img.shape)
        dog_img = dog_img / 255.
        dog_img = dog_img.reshape((1,224,224,3))
        dog_img_array = np.concatenate([dog_img_array, dog_img], axis = 0)
        dog_img_.close()
    
    if re.search('cat', img):
        
        cat_img_ = Image.open(img)
        cat_img_ = cat_img_.resize((224, 224))
        cat_img = np.array(cat_img_)
        cat_img = cat_img / 255.
        cat_img = cat_img.reshape((1,224,224,3))
        cat_img_array = np.concatenate([cat_img_array, cat_img], axis = 0)
        cat_img_.close()

In [ ]:
print('dog_image:{}  cat_image:{}'.format(dog_img_array.shape, cat_img_array.shape))

In [ ]:
X_test = np.concatenate((cat_img_array, dog_img_array), axis=0)
X_test.shape

In [ ]:
# 精度上がらず

model.predict(X_test, batch_size=None, verbose=1, steps=None)

In [ ]:
img